In [2]:
# ============================
#   ONE-CELL FINAL DEMO (WITH PROGRESS)
# ============================

import os, sys, subprocess, random, re, base64, time
from pathlib import Path
import numpy as np

# Force HF progress bars ON (sometimes notebooks hide them)
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "0"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"  # faster downloads if available

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

# ============================
# ✅ DEPENDENCY INSTALL
# ============================
print("📦 Installing dependencies...")
pip_install([
    "huggingface_hub>=0.23.0",
    "transformers>=4.40.0",
    "torch",
    "librosa",
    "opencv-python",
    "tensorflow",
    "ipython",
    "imageio-ffmpeg"
])
print("✅ Dependencies installed.\n")

import torch, librosa, cv2
from huggingface_hub import hf_hub_download, HfApi
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
from tensorflow.keras.models import load_model
from IPython.display import HTML, display
import imageio_ffmpeg

MODEL_REPO = "Seldarzu/crema_d_MER_arzuSeldaAvci_ayazAktas"
DATA_REPO  = "Seldarzu/crema_d_Fusion_Test_Videos"

AUDIO_CKPT_IN_REPO  = "checkpoints/best_wav2vec2_speaker_indep.pt"
VISUAL_CKPT_IN_REPO = "checkpoints/best_final_model.keras"

CLASS_LABELS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
EMO_MAP = {"ANG":"angry","DIS":"disgust","FEA":"fear","HAP":"happy","NEU":"neutral","SAD":"sad"}

N_DEMOS = 10
W_FINAL = 0.65
TARGET_SR = 16000
MAX_SEC = 3.5

SAMPLE_EVERY_N_FRAMES = 2
MAX_FACE_SAMPLES = 30
FACE_MIN_SIZE = (60, 60)

TMP = Path("_tmp_demo")
TMP.mkdir(exist_ok=True)

FFMPEG = imageio_ffmpeg.get_ffmpeg_exe()

def topk(proba, k=3):
    idx = np.argsort(-proba)[:k]
    return [(CLASS_LABELS[i], float(proba[i])) for i in idx]

def fuse_probs(p_audio, p_visual, w_audio):
    p = w_audio * p_audio + (1.0 - w_audio) * p_visual
    return p / (p.sum() + 1e-9)

def parse_gt_from_filename(fname: str):
    m = re.search(r"_(ANG|DIS|FEA|HAP|NEU|SAD)_", fname)
    return EMO_MAP.get(m.group(1), "unknown") if m else "unknown"

def ensure_browser_mp4(in_path: Path) -> Path:
    out_path = TMP / (in_path.stem + "_browser.mp4")
    if out_path.exists() and out_path.stat().st_size > 0:
        return out_path
    cmd = [
        FFMPEG, "-y",
        "-i", str(in_path),
        "-c:v", "libx264", "-pix_fmt", "yuv420p",
        "-preset", "veryfast", "-crf", "23",
        "-c:a", "aac", "-b:a", "128k",
        "-movflags", "+faststart",
        str(out_path)
    ]
    r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if r.returncode != 0 or (not out_path.exists()) or out_path.stat().st_size == 0:
        print("❌ browser mp4 encode failed:", in_path.name)
        print(r.stderr.decode("utf-8")[-1200:])
        return in_path
    return out_path

def show_video_safe(path: Path, width=720, autoplay=False, loop=False, muted=True):
    p = ensure_browser_mp4(path)
    data = p.read_bytes()
    b64 = base64.b64encode(data).decode("utf-8")
    auto = "autoplay" if autoplay else ""
    lp = "loop" if loop else ""
    mut = "muted" if muted else ""
    html = f"""
    <video width="{width}" controls {auto} {lp} {mut}>
      <source src="data:video/mp4;base64,{b64}" type="video/mp4">
      Your browser does not support the video tag.
    </video>
    """
    display(HTML(html))

# ----------------------------
# ✅ DOWNLOAD MODELS (with progress logs)
# ----------------------------
print("⬇️ Downloading AUDIO checkpoint...")
audio_ckpt_path = hf_hub_download(repo_id=MODEL_REPO, filename=AUDIO_CKPT_IN_REPO, repo_type="model")
print("✅ AUDIO ckpt ready:", audio_ckpt_path, "\n")

print("⬇️ Downloading VISUAL checkpoint...")
visual_ckpt_path = hf_hub_download(repo_id=MODEL_REPO, filename=VISUAL_CKPT_IN_REPO, repo_type="model")
print("✅ VISUAL ckpt ready:", visual_ckpt_path, "\n")

# ----------------------------
# ✅ DOWNLOAD DEMOS (with per-file counter)
# ----------------------------
print("📚 Listing dataset files...")
api = HfApi()
files = api.list_repo_files(repo_id=DATA_REPO, repo_type="dataset")
video_files = [f for f in files if f.startswith("demos/") and re.search(r"\.(mp4|m4v|mov|avi|flv)$", f, re.IGNORECASE)]
if not video_files:
    raise RuntimeError(f"❌ No videos found under demos/ in {DATA_REPO}")

random.shuffle(video_files)
video_files = video_files[:min(N_DEMOS, len(video_files))]

raw_demo_paths = []
print(f"⬇️ Downloading demo videos: {len(video_files)} files")
for idx, vf in enumerate(video_files, 1):
    print(f"   [{idx}/{len(video_files)}] downloading {vf} ...")
    p = Path(hf_hub_download(repo_id=DATA_REPO, filename=vf, repo_type="dataset"))
    raw_demo_paths.append(p)
print("✅ All demo videos downloaded:", [p.name for p in raw_demo_paths], "\n")

# ----------------------------
# ✅ LOAD AUDIO MODEL
# ----------------------------
print("🧠 Loading AUDIO model...")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
audio_model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base-960h",
    num_labels=len(CLASS_LABELS),
    problem_type="single_label_classification"
)

state = torch.load(audio_ckpt_path, map_location="cpu")
if isinstance(state, dict) and "state_dict" in state:
    state = state["state_dict"]
if isinstance(state, dict) and "model" in state:
    state = state["model"]
audio_model.load_state_dict(state, strict=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
audio_model.to(device).eval()
print("✅ Audio model on:", device, "\n")

# ----------------------------
# ✅ LOAD VISUAL MODEL
# ----------------------------
print("🧠 Loading VISUAL model...")
visual_model = load_model(visual_ckpt_path, compile=False)
print("✅ Visual model loaded.\n")

# ----------------------------
# Audio extract + proba
# ----------------------------
def extract_wav(video_path: Path, out_wav: Path, sr=16000):
    cmd = [FFMPEG, "-y", "-i", str(video_path), "-vn", "-ac", "1", "-ar", str(sr), "-f", "wav", str(out_wav)]
    r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if r.returncode != 0 or (not out_wav.exists()) or out_wav.stat().st_size == 0:
        print("❌ audio extract failed:", video_path.name)
        print(r.stderr.decode("utf-8")[-800:])
        return None
    return out_wav

def get_audio_proba(video_path: Path, max_sec=3.5):
    wav_path = TMP / (video_path.stem + f"_{TARGET_SR}.wav")
    if not wav_path.exists():
        extract_wav(video_path, wav_path, sr=TARGET_SR)

    max_len = int(TARGET_SR * max_sec)
    if wav_path.exists():
        audio, _ = librosa.load(str(wav_path), sr=TARGET_SR)
    else:
        audio = np.zeros(max_len, dtype=np.float32)

    if len(audio) > max_len: audio = audio[:max_len]
    else: audio = np.pad(audio, (0, max_len - len(audio)))
    audio = audio.astype(np.float32)

    inputs = processor([audio], sampling_rate=TARGET_SR, return_tensors="pt", padding=True, return_attention_mask=True)
    x  = inputs.input_values.to(device)
    am = inputs.attention_mask.to(device)

    with torch.no_grad():
        logits = audio_model(input_values=x, attention_mask=am).logits
        p = torch.softmax(logits, dim=-1)[0].detach().cpu().numpy()
    return p / (p.sum() + 1e-9)

# ----------------------------
# Visual + annotate
# ----------------------------
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def preprocess_face_bgr(face_bgr, size=(224,224)):
    x = cv2.resize(face_bgr, size)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.astype(np.float32) / 255.0
    return np.expand_dims(x, axis=0)

def put_text_shadow(img, text, org, font, scale, color, thickness, shadow_color=(255,255,255)):
    x, y = org
    cv2.putText(img, text, (x+1, y+1), font, scale, shadow_color, thickness+2, cv2.LINE_AA)
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

def annotate_video(video_path: Path, gt_label: str, pa: np.ndarray):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None, None, {"reason":"open failed"}

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps is None or fps <= 1:
        fps = 25

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    raw_out = TMP / f"{video_path.stem}_annot_raw.mp4"
    writer = cv2.VideoWriter(str(raw_out), cv2.VideoWriter_fourcc(*"mp4v"), fps, (W, H))

    audio_pred = CLASS_LABELS[int(np.argmax(pa))]
    audio_conf = float(pa.max())

    preds_visual = []
    frames = 0
    faces_seen = 0
    font = cv2.FONT_HERSHEY_SIMPLEX

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frames += 1

        line2_y = H - 20
        line1_y = H - 50
        x0 = 10

        put_text_shadow(frame, f"REAL: {gt_label}", (x0, line1_y), font, 0.75, (0,0,0), 2)
        put_text_shadow(frame, f"AUDIO: {audio_pred} ({audio_conf:.2f})", (x0, line2_y), font, 0.75, (0,0,0), 2)

        if frames % SAMPLE_EVERY_N_FRAMES == 0 and faces_seen < MAX_FACE_SAMPLES:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=FACE_MIN_SIZE)

            if len(faces) > 0:
                (x,y,w,h) = sorted(faces, key=lambda t: t[2]*t[3], reverse=True)[0]
                face = frame[y:y+h, x:x+w]

                xin = preprocess_face_bgr(face, (224,224))
                pv = visual_model.predict(xin, verbose=0)[0]
                pv = np.array(pv, dtype=np.float32)
                pv = pv / (pv.sum() + 1e-9)

                preds_visual.append(pv)
                faces_seen += 1

                vis_pred = CLASS_LABELS[int(np.argmax(pv))]
                vis_conf = float(pv.max())

                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,0), 2)
                label_y = max(25, y-10)
                put_text_shadow(frame, f"VIS: {vis_pred} ({vis_conf:.2f})", (x, label_y), font, 0.7, (0,0,0), 2)

        writer.write(frame)

    cap.release()
    writer.release()

    if len(preds_visual) == 0:
        pv_mean = None
        meta = {"frames":frames, "faces_seen":0, "reason":"no face detected"}
    else:
        pv_mean = np.mean(np.stack(preds_visual, axis=0), axis=0)
        pv_mean = pv_mean / (pv_mean.sum() + 1e-9)
        meta = {"frames":frames, "faces_seen":faces_seen}

    annotated_h264 = ensure_browser_mp4(raw_out)
    return annotated_h264, pv_mean, meta

# ----------------------------
# RUN
# ----------------------------
print(f"🎬 FINAL DEMO | n={len(raw_demo_paths)} | w={W_FINAL} | device={device}\n")

for i, raw_path in enumerate(raw_demo_paths, 1):
    gt = parse_gt_from_filename(raw_path.name)
    print("\n" + "="*85)
    print(f"[{i}] {raw_path.name} | REAL={gt}")

    input_mp4 = ensure_browser_mp4(raw_path)

    pa = get_audio_proba(input_mp4, max_sec=MAX_SEC)
    print("🎧 AUDIO :", CLASS_LABELS[int(np.argmax(pa))], "| conf:", float(pa.max()))
    print("   top-3:", topk(pa, 3))

    out_vid, pv, meta = annotate_video(input_mp4, gt, pa)
    if out_vid is not None:
        print("🎞️ Annotated video (playable):")
        show_video_safe(out_vid, width=720)

    if pv is None:
        print("🎥 VISUAL: NO FACE -> fallback audio")
        print("   meta:", meta)
        pf = pa
    else:
        print("🎥 VISUAL:", CLASS_LABELS[int(np.argmax(pv))], "| conf:", float(pv.max()))
        print("   top-3:", topk(pv, 3))
        print("   meta:", meta)
        pf = fuse_probs(pa, pv, W_FINAL)

    print("🤝 FUSION:", CLASS_LABELS[int(np.argmax(pf))], "| conf:", float(pf.max()))
    print("   top-3:", topk(pf, 3))

print("\n✅ DONE.")


📦 Installing dependencies...
✅ Dependencies installed.

⬇️ Downloading AUDIO checkpoint...
✅ AUDIO ckpt ready: /root/.cache/huggingface/hub/models--Seldarzu--crema_d_MER_arzuSeldaAvci_ayazAktas/snapshots/b185d05c1e9f150232d729fc6a3ef63010734520/checkpoints/best_wav2vec2_speaker_indep.pt 

⬇️ Downloading VISUAL checkpoint...
✅ VISUAL ckpt ready: /root/.cache/huggingface/hub/models--Seldarzu--crema_d_MER_arzuSeldaAvci_ayazAktas/snapshots/b185d05c1e9f150232d729fc6a3ef63010734520/checkpoints/best_final_model.keras 

📚 Listing dataset files...
⬇️ Downloading demo videos: 10 files
   [1/10] downloading demos/1052_TIE_FEA_XX.flv ...


demos/1052_TIE_FEA_XX.flv:   0%|          | 0.00/412k [00:00<?, ?B/s]

   [2/10] downloading demos/1085_TIE_ANG_XX.flv ...
   [3/10] downloading demos/1021_IOM_SAD_XX.flv ...
   [4/10] downloading demos/1083_DFA_ANG_XX.flv ...
   [5/10] downloading demos/1021_IWW_NEU_XX.flv ...
   [6/10] downloading demos/1024_DFA_HAP_XX.flv ...


demos/1024_DFA_HAP_XX.flv:   0%|          | 0.00/249k [00:00<?, ?B/s]

   [7/10] downloading demos/1003_IEO_NEU_XX.flv ...


...

🎥 VISUAL: happy | conf: 0.7755913138389587
   top-3: [('happy', 0.7755913138389587), ('neutral', 0.18670091032981873), ('sad', 0.03184395655989647)]
   meta: {'frames': 59, 'faces_seen': 29}
🤝 FUSION: happy | conf: 0.4675883948802948
   top-3: [('happy', 0.4675883948802948), ('fear', 0.44276243448257446), ('neutral', 0.06592415273189545)]

[7] 1003_IEO_NEU_XX.flv | REAL=neutral
🎧 AUDIO : neutral | conf: 0.9724581837654114
   top-3: [('neutral', 0.9724581837654114), ('happy', 0.015347626991569996), ('angry', 0.004461556673049927)]
🎞️ Annotated video (playable):


🎥 VISUAL: happy | conf: 0.9906595945358276
   top-3: [('happy', 0.9906595945358276), ('fear', 0.005593754816800356), ('neutral', 0.002377997385337949)]
   meta: {'frames': 69, 'faces_seen': 30}
🤝 FUSION: neutral | conf: 0.6329300999641418
   top-3: [('neutral', 0.6329300999641418), ('happy', 0.356706827878952), ('angry', 0.0032993361819535494)]

[8] 1022_IOM_HAP_XX.flv | REAL=happy
🎧 AUDIO : happy | conf: 0.983936071395874
   top-3: [('happy', 0.983936071395874), ('fear', 0.009126401506364346), ('disgust', 0.0032283731270581484)]
🎞️ Annotated video (playable):


🎥 VISUAL: happy | conf: 0.7379284501075745
   top-3: [('happy', 0.7379284501075745), ('disgust', 0.14659111201763153), ('neutral', 0.08181063085794449)]
   meta: {'frames': 74, 'faces_seen': 30}
🤝 FUSION: happy | conf: 0.8978334069252014
   top-3: [('happy', 0.8978334069252014), ('disgust', 0.05340533331036568), ('neutral', 0.030049823224544525)]

[9] 1083_IOM_HAP_XX.flv | REAL=happy
🎧 AUDIO : happy | conf: 0.6661442518234253
   top-3: [('happy', 0.6661442518234253), ('fear', 0.1357572227716446), ('angry', 0.11298135668039322)]
🎞️ Annotated video (playable):


🎥 VISUAL: happy | conf: 0.36216863989830017
   top-3: [('happy', 0.36216863989830017), ('neutral', 0.3606283962726593), ('sad', 0.2746923863887787)]
   meta: {'frames': 63, 'faces_seen': 30}
🤝 FUSION: happy | conf: 0.5597527623176575
   top-3: [('happy', 0.5597527623176575), ('neutral', 0.16778166592121124), ('sad', 0.10185901820659637)]

[10] 1021_ITH_SAD_XX.flv | REAL=sad
🎧 AUDIO : sad | conf: 0.8182478547096252
   top-3: [('sad', 0.8182478547096252), ('fear', 0.1118599995970726), ('neutral', 0.062022414058446884)]
🎞️ Annotated video (playable):


🎥 VISUAL: neutral | conf: 0.8112834692001343
   top-3: [('neutral', 0.8112834692001343), ('happy', 0.13175241649150848), ('fear', 0.04710298031568527)]
   meta: {'frames': 90, 'faces_seen': 30}
🤝 FUSION: sad | conf: 0.5336931347846985
   top-3: [('sad', 0.5336931347846985), ('neutral', 0.3242637515068054), ('fear', 0.08919503539800644)]

✅ DONE.
